<b>The goal of this project is to build a trading analysis system to signal when to buy and when to sell stocks in your investment portfolio. The strategy is based mainly on volume changes and if there is a big volume increase then we are assuming it is actually a good price to enter, its good to buy.</b>

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import math
import warnings

In [2]:
warnings.filterwarnings("ignore")

In [3]:
prices = pd.read_csv("adjclose.csv", index_col="Date", parse_dates=True)
volumechanges = pd.read_csv("volume.csv", index_col="Date", parse_dates=True).pct_change()*100

In [4]:
today = dt.date(2017, 1, 2)
simend = dt.date(2020, 10, 1)
tickers = []
transactionid = 0
money = 1000000
portfolio = {}
activelog = []
transactionlog = []

In [5]:
def getprice(date, ticker):
    global prices
    try:
        price = prices.loc[str(date)][ticker]
        return price
    except Exception as e:
        return None

In [6]:
def transaction(id, ticker, amount, price, type,profit):
    global transactionid
    if type == "buy":
        exp_date = today + dt.timedelta(days=14)
        transactionid += 1
    else:
        exp_date = today
    if type == "sell":
        data = {"id": id, "ticker": ticker, "amount": amount, "price": price, "date": today, "type": type,
                "exp_date": exp_date, "profit": profit}
    elif type == "buy":
        data = {"id": transactionid, "ticker": ticker, "amount": amount, "price": price, "date": today, "type": type,
                "exp_date": exp_date, "profit": profit}
        activelog.append(data)
    transactionlog.append(data)

In [7]:
def buy(interestlst, allocated_money):
    global money, portfolio
    for item in interestlst:
        price = getprice(today, item)
        if not pd.isnull(price):
            quantity = math.floor(allocated_money/price)
            money -= quantity*price
            portfolio[item] += quantity
            transaction(0, item, quantity, price, "buy", 0)

In [8]:
def sell():
    global money, portfolio, prices, today
    itemstoremove = []
    for i in range(len(activelog)):
        log = activelog[i]
        if log["exp_date"] <= today and log["type"] == "buy":
            tickprice = getprice(today, log["ticker"])
            if not pd.isnull(tickprice):
                money += log["amount"]*tickprice
                portfolio[log["ticker"]] -= log["amount"]
                profit = log["amount"]*tickprice - log["amount"]*log["price"]
                transaction(log["id"], log["ticker"], log["amount"], tickprice, "sell",profit)
                itemstoremove.append(i)
            else:
                log["exp_date"] += dt.timedelta(days=1)
    itemstoremove.reverse()
    for elem in itemstoremove:
        activelog.remove(activelog[elem])

In [9]:
def simulation():
    global today, volumechanges, money
    start_date = today - dt.timedelta(days=14)
    series = volumechanges.loc[start_date:today].mean()
    interestlst = series[series > 100].index.tolist()
    sell()
    if len(interestlst) > 0:
        #moneyToAllocate = 500000/len(interestlst)
        moneyToAllocate = currentvalue()/(2*len(interestlst))
        buy(interestlst, moneyToAllocate)

In [10]:
def getindices():
    global tickers
    f = open("symbols.txt", "r")
    for line in f:
        tickers.append(line.strip())
    f.close()

In [11]:
def tradingday():
    global prices, today
    return np.datetime64(today) in list(prices.index.values)

In [12]:
def currentvalue():
    global money, portfolio, today, prices
    value = money
    for ticker in tickers:
        tickprice = getprice(today, ticker)
        if not pd.isnull(tickprice):
            value += portfolio[ticker]*tickprice
    return int(value*100)/100

In [13]:
def main():
    global today
    getindices()
    for ticker in tickers:
        portfolio[ticker] = 0
    while today < simend:
        while not tradingday():
            today += dt.timedelta(days=1)
        simulation()
        currentpvalue = currentvalue()
        print(currentpvalue, today)
        today += dt.timedelta(days=7)

    df = pd.DataFrame(transactionlog)
    df.to_csv('transactions.csv',index=False)

main()

1000000.0 2017-01-03
1039602.94 2017-01-10
1042190.82 2017-01-17
1049431.6 2017-01-24
1058696.44 2017-01-31
1090651.49 2017-02-07
1165307.09 2017-02-14
1182295.01 2017-02-21
1148374.64 2017-02-28
1149629.31 2017-03-07
1165870.48 2017-03-14
1181131.01 2017-03-21
1200577.68 2017-03-28
1207567.02 2017-04-04
1204550.33 2017-04-11
1195417.45 2017-04-18
1216132.28 2017-04-25
1239269.6 2017-05-02
1222061.78 2017-05-09
1204630.82 2017-05-16
1174954.45 2017-05-23
1196369.61 2017-05-30
1256062.62 2017-06-06
1247171.88 2017-06-13
1269354.17 2017-06-20
1268184.45 2017-06-27
1291162.54 2017-07-05
1300482.9 2017-07-12
1327545.9 2017-07-19
1334101.82 2017-07-26
1375401.66 2017-08-02
1373009.22 2017-08-09
1378542.02 2017-08-16
1404773.81 2017-08-23
1404773.81 2017-08-30
1431452.36 2017-09-06
1425032.56 2017-09-13
1388914.66 2017-09-20
1506875.49 2017-09-27
1538767.18 2017-10-04
1531569.9 2017-10-11
1495324.65 2017-10-18
1475784.79 2017-10-25
1471947.43 2017-11-01
1471947.43 2017-11-08
1443570.27 2017-

In [14]:
df = pd.read_csv('transactions.csv')

In [15]:
df.head()

,id,ticker,amount,price,date,type,exp_date,profit
0,1,HWM,26089,19.164694,2017-01-03,buy,2017-01-17,0.000000
1,2,IDXX,1435,120.709999,2017-01-10,buy,2017-01-24,0.000000
2,3,WMB,7549,22.951468,2017-01-10,buy,2017-01-24,0.000000
3,4,IT,1774,97.620003,2017-01-10,buy,2017-01-24,0.000000
4,1,HWM,26089,20.712650,2017-01-17,sell,2017-01-17,40384.636259


In [16]:
df.groupby('ticker')['profit'].sum().sort_values(ascending=False).to_csv("best_tickers.csv")

In [17]:
df[df['type']=='sell'].groupby('ticker')['profit'].count().sort_values(ascending=False)

ticker
AMCR    108
USD      10
IT        4
CARR      4
OTIS      4
       ... 
BMY       1
O         1
BSX       1
CDNS      1
INFO      1
Name: profit, Length: 97, dtype: int64

In [18]:
df.loc[df['profit'] > 0, 'profitable'] = 1 
df.loc[df['profit'] <= 0, 'profitable'] = 0 

In [19]:
df['profitable'] = df['profit'].apply(lambda x: 1 if x > 0 else 0)

In [20]:
df.head(10)

,id,ticker,amount,price,date,type,exp_date,profit,profitable
0,1,HWM,26089,19.164694,2017-01-03,buy,2017-01-17,0.000000,0
1,2,IDXX,1435,120.709999,2017-01-10,buy,2017-01-24,0.000000,0
2,3,WMB,7549,22.951468,2017-01-10,buy,2017-01-24,0.000000,0
3,4,IT,1774,97.620003,2017-01-10,buy,2017-01-24,0.000000,0
4,1,HWM,26089,20.712650,2017-01-17,sell,2017-01-17,40384.636259,1
5,5,IDXX,1447,120.010002,2017-01-17,buy,2017-01-31,0.000000,0
6,6,WMB,7584,22.903147,2017-01-17,buy,2017-01-31,0.000000,0
7,7,ANET,1968,88.230003,2017-01-17,buy,2017-01-31,0.000000,0
8,2,IDXX,1435,118.510002,2017-01-24,sell,2017-01-24,-3156.995621,0
9,3,WMB,7549,22.935362,2017-01-24,sell,2017-01-24,-121.581566,0


In [21]:
df[df['type']=='sell'].groupby('ticker')['profitable'].mean().sort_values(ascending=False).to_csv('successratio.csv')

In [22]:
df[df['type']=='sell'].groupby('ticker').agg({'profitable':['sum', 'mean'], 
                         'profit':['sum','min','max','mean', lambda x: x.max() - x.min()] 
                         }).sort_values([('profit', 'sum')], ascending=False)

profitable                   profit                                \
              sum      mean            sum            min            max   
ticker                                                                     
AMCR           51  0.472222  199809.264024  -37665.595151   57996.819256   
NLSN            3  1.000000  145119.954597     648.056168   78855.430077   
LB              1  1.000000  132607.549192  132607.549192  132607.549192   
CARR            2  0.500000  115380.215464 -102253.621896  134064.871373   
EFX             2  1.000000  111226.955719   53618.246429   57608.709290   
...           ...       ...            ...            ...            ...   
DXCM            0  0.000000  -45551.669025  -25190.531696  -20361.137329   
INCY            0  0.000000  -58819.199638  -46949.102383  -11870.097256   
DXC             0  0.000000 -103105.468224  -58326.163376  -44779.304848   
FE              1  0.333333 -108031.141184 -133698.894131   37338.482073   
CNC             1  0.500000 -123702.779144 -193002.829880   69300.050735   

                                      
                 mean     <lambda_0>  
ticker                                
AMCR      1850.085778   95662.414407  
NLSN     48373.318199   78207.373909  
LB      132607.549192       0.000000  
CARR     28845.053866  236318.493269  
EFX      55613.477859    3990.462860  
...               ...            ...  
DXCM    -22775.834513    4829.394367  
INCY    -29409.599819   35079.005127  
DXC     -51552.734112   13546.858528  
FE      -36010.380395  171037.376204  
CNC     -61851.389572  262302.880615  

[97 rows x 7 columns]

In [23]:
df.groupby('ticker').head()

,id,ticker,amount,price,date,type,exp_date,profit,profitable
0,1,HWM,26089,19.164694,2017-01-03,buy,2017-01-17,0.000000,0
1,2,IDXX,1435,120.709999,2017-01-10,buy,2017-01-24,0.000000,0
2,3,WMB,7549,22.951468,2017-01-10,buy,2017-01-24,0.000000,0
3,4,IT,1774,97.620003,2017-01-10,buy,2017-01-24,0.000000,0
4,1,HWM,26089,20.712650,2017-01-17,sell,2017-01-17,40384.636259,1
...,...,...,...,...,...,...,...,...,...
647,324,LB,35937,30.620001,2020-08-25,sell,2020-08-25,132607.549192,1
648,325,CTLT,5973,85.169998,2020-09-22,buy,2020-10-06,0.000000,0
649,326,TER,6645,76.559998,2020-09-22,buy,2020-10-06,0.000000,0
650,327,CTLT,6037,85.190002,2020-09-29,buy,2020-10-13,0.000000,0
